In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from datetime import datetime

import PIL
# Rmb to run "pip install pillow"
# from ResNet import Bottleneck, ResNet, ResNet50

In [ ]:
import os
from os import listdir

In [ ]:
# Folders
train_FAKE_folder_path = "C:/Users/tan_l/Workspace/CS5242/Group Project/Data/Actual Subset/train/fake"
train_REAL_folder_path = "C:/Users/tan_l/Workspace/CS5242/Group Project/Data/Actual Subset/train/real"

test_FAKE_folder_path = "C:/Users/tan_l/Workspace/CS5242/Group Project/Data/Actual Subset/test/fake"
test_REAL_folder_path = "C:/Users/tan_l/Workspace/CS5242/Group Project/Data/Actual Subset/test/real"

## Prepare TRAIN and TEST Tensor dataset (Let label for Real be 1 and label for Fake be 0)

In [ ]:
def ignore_alpha(image):
    if image.mode == 'RGBA':
        r, g, b, _ = image.split()
        return PIL.Image.merge("RGB", (r, g, b))
    elif image.mode == 'RGB':
        return image
    else:
        return image.convert("RGB")

transform_train_V1 = transforms.Compose([
    ignore_alpha,
    transforms.Resize((224,224)),
    transforms.CenterCrop((224,224)),
    transforms.RandomHorizontalFlip(), # To introduce image orientation variation
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

])

transform_test_V1 = transforms.Compose([
    ignore_alpha,
    transforms.Resize((224, 224)),
    transforms.CenterCrop((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [ ]:
train_tensor_list =[]
#Format = list of tuples with each tuple in this format ((tensor version of image), label)

# FAKE portion
for images in os.listdir(train_FAKE_folder_path):
    # print(images)
    # if images.endswith(".png"):
    img = PIL.Image.open(f"{train_FAKE_folder_path}/%s" % images)
    # train_tensor_list.append(transform_train_V1(img))
    train_tensor_list.append((transform_train_V1(img), 0))
# REAL portion
for images in os.listdir(train_REAL_folder_path):
    # if images.endswith(".png"):
    img = PIL.Image.open(f"{train_REAL_folder_path}/%s" % images)
    # train_tensor_list.append(transform_train_V1(img))
    train_tensor_list.append((transform_train_V1(img), 1))

In [ ]:
test_tensor_list =[]
#Format = list of tuples with each tuple in this format ((tensor version of image), label)

# FAKE portion
for images in os.listdir(test_FAKE_folder_path):
    # print(images)
    img = PIL.Image.open(f"{test_FAKE_folder_path}/%s" % images)
    # test_tensor_list.append(transform_test_V1(img))
    test_tensor_list.append((transform_test_V1(img),0))
# REAL portion

for images in os.listdir(test_REAL_folder_path):
    # print(images)
    img = PIL.Image.open(f"{test_REAL_folder_path}/%s" % images)
    # test_tensor_list.append(transform_test_V1(img))
    test_tensor_list.append((transform_test_V1(img),1))

In [ ]:
print(len(train_tensor_list))
print(len(test_tensor_list))

3300
1200


In [ ]:
print(len(train_tensor_list))
print(len(test_tensor_list))

3300
1200


In [ ]:
## LOAD DATA to trainloader and testloader

# train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

trainloader = torch.utils.data.DataLoader(train_tensor_list, batch_size=128, shuffle=True, num_workers=2)

# test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

testloader = torch.utils.data.DataLoader(test_tensor_list, batch_size=128,shuffle=False, num_workers=2)

for X, y in trainloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([128, 3, 224, 224])
Shape of y: torch.Size([128]) torch.int64


### DenseNET Model 121

In [ ]:
import torch

# input_batch = torch.stack(train_tensor_list)
# print(input_batch)

# device = (
#     "cuda"
#     if torch.cuda.is_available()
#     else "mps"
#     if torch.backends.mps.is_available()
#     else "cpu"
# )

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)
# # or any of these variants
# # model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet169', pretrained=True)
# # model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet201', pretrained=True)
# # model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet161', pretrained=True)
# model.eval()

# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model.eval()

if torch.cuda.is_available():
    model.cuda()

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# with torch.no_grad():
#     output = model(input_batch)
# # Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
# print(output[0])
# # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
# probabilities = torch.nn.functional.softmax(output[0], dim=0)
# print(probabilities)

Using cuda device


Using cache found in C:\Users\tan_l/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\tan_l\anaconda3\envs\PyTorch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\tan_l\anaconda3\envs\PyTorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import time

optimizer_lr =[1e-2,1e-3,1e-4]

for j in optimizer_lr:
    model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)
    model.train()
    if torch.cuda.is_available():
        model.cuda()


    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=j)

    def train(dataloader, model, loss_fn, optimizer):
        size = len(dataloader.dataset)
        model.train()
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            # Compute prediction error
            pred = model(X)
            loss = loss_fn(pred, y)

            # Backpropagation
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if batch % 100 == 0:
                loss, current = loss.item(), (batch + 1) * len(X)
                # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        return loss

    def test(dataloader, model, loss_fn):
        size = len(dataloader.dataset)
        num_batches = len(dataloader)
        model.eval()
        test_loss, correct = 0, 0
        with torch.no_grad():
            for X, y in dataloader:
                X, y = X.to(device), y.to(device)
                pred = model(X)
                test_loss += loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        # print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
        return (100*correct) , test_loss

    epochs = 10
    print("Started:", datetime.now())

    start = time.time()

    for t in range(epochs):

        loss = train(trainloader, model, loss_fn, optimizer)
        accuracy , avg_loss = test(testloader, model, loss_fn)
        if t == (epochs-1):
            end = time.time()
            print("Learning Rate:", j)
            print("Start Time :", start)
            print("End Time :", end)
            print("Time Taken:", end - start)
            print(f"Epoch {t+1}\n-------------------------------")
            print("Train Loss :", loss)
            print("Test Avg Loss :", avg_loss)
            print("Test Accuracy :", accuracy)
            print("\n-------------------------------")
print("Done!")

Using cache found in C:\Users\tan_l/.cache\torch\hub\pytorch_vision_v0.10.0


Started: 2024-04-28 22:37:36.115947
Learning Rate: 0.01
Start Time : 1714315056.1159477
End Time : 1714320164.8517566
Time Taken: 5108.735808849335
Epoch 10
-------------------------------
Train Loss : tensor(0.0174, device='cuda:0', grad_fn=<NllLossBackward0>)
Test Avg Loss : 0.3399777770042419
Test Accuracy : 85.91666666666666

-------------------------------


Using cache found in C:\Users\tan_l/.cache\torch\hub\pytorch_vision_v0.10.0


Started: 2024-04-29 00:02:45.085735
Learning Rate: 0.001
Start Time : 1714320165.0857353
End Time : 1714325247.4073293
Time Taken: 5082.321593999863
Epoch 10
-------------------------------
Train Loss : tensor(0.4258, device='cuda:0', grad_fn=<NllLossBackward0>)
Test Avg Loss : 0.49918232560157777
Test Accuracy : 78.08333333333334

-------------------------------
Started: 2024-04-29 01:27:27.578346


Using cache found in C:\Users\tan_l/.cache\torch\hub\pytorch_vision_v0.10.0


Learning Rate: 0.0001
Start Time : 1714325247.578347
End Time : 1714330320.2631247
Time Taken: 5072.684777736664
Epoch 10
-------------------------------
Train Loss : tensor(6.8101, device='cuda:0', grad_fn=<NllLossBackward0>)
Test Avg Loss : 7.172862195968628
Test Accuracy : 3.0833333333333335

-------------------------------
Done!


### DenseNET Model 169

In [ ]:
import torch

# input_batch = torch.stack(train_tensor_list)
# print(input_batch)

# device = (
#     "cuda"
#     if torch.cuda.is_available()
#     else "mps"
#     if torch.backends.mps.is_available()
#     else "cpu"
# )

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

# model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)
# # or any of these variants
model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet169', pretrained=True)
# # model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet201', pretrained=True)
# # model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet161', pretrained=True)
# model.eval()

# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model.eval()

if torch.cuda.is_available():
    model.cuda()

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# with torch.no_grad():
#     output = model(input_batch)
# # Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
# print(output[0])
# # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
# probabilities = torch.nn.functional.softmax(output[0], dim=0)
# print(probabilities)

Using cuda device


Using cache found in C:\Users\tan_l/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\tan_l\anaconda3\envs\PyTorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet169_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet169_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to C:\Users\tan_l/.cache\torch\hub\checkpoints\densenet169-b2777c0a.pth
100%|██████████| 54.7M/54.7M [00:01<00:00, 41.2MB/s]


In [ ]:
optimizer_lr =[1e-2,1e-3,1e-4]

for j in optimizer_lr:
    model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet169', pretrained=True)
    model.train()
    if torch.cuda.is_available():
        model.cuda()


    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=j)

    def train(dataloader, model, loss_fn, optimizer):
        size = len(dataloader.dataset)
        model.train()
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            # Compute prediction error
            pred = model(X)
            loss = loss_fn(pred, y)

            # Backpropagation
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if batch % 100 == 0:
                loss, current = loss.item(), (batch + 1) * len(X)
                # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        return loss

    def test(dataloader, model, loss_fn):
        size = len(dataloader.dataset)
        num_batches = len(dataloader)
        model.eval()
        test_loss, correct = 0, 0
        with torch.no_grad():
            for X, y in dataloader:
                X, y = X.to(device), y.to(device)
                pred = model(X)
                test_loss += loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        # print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
        return (100*correct) , test_loss

    epochs = 10
    print("Started:", datetime.now())

    start = time.time()

    for t in range(epochs):

        loss = train(trainloader, model, loss_fn, optimizer)
        accuracy , avg_loss = test(testloader, model, loss_fn)
        if t == (epochs-1):
            end = time.time()
            print("Learning Rate:", j)
            print("Start Time :", start)
            print("End Time :", end)
            print("Time Taken:", end - start)
            print(f"Epoch {t+1}\n-------------------------------")
            print("Train Loss :", loss)
            print("Test Avg Loss :", avg_loss)
            print("Test Accuracy :", accuracy)
            print("\n-------------------------------")
print("Done!")

Using cache found in C:\Users\tan_l/.cache\torch\hub\pytorch_vision_v0.10.0


Started: 2024-04-29 02:52:03.553995
Learning Rate: 0.01
Start Time : 1714330323.553995
End Time : 1714337113.6740913
Time Taken: 6790.120096445084
Epoch 10
-------------------------------
Train Loss : tensor(0.0159, device='cuda:0', grad_fn=<NllLossBackward0>)
Test Avg Loss : 0.34774726927280425
Test Accuracy : 85.5

-------------------------------


Using cache found in C:\Users\tan_l/.cache\torch\hub\pytorch_vision_v0.10.0


Started: 2024-04-29 04:45:13.921148
Learning Rate: 0.001
Start Time : 1714337113.921148
End Time : 1714343964.4019084
Time Taken: 6850.480760335922
Epoch 10
-------------------------------
Train Loss : tensor(0.3008, device='cuda:0', grad_fn=<NllLossBackward0>)
Test Avg Loss : 0.48998236656188965
Test Accuracy : 78.66666666666666

-------------------------------


Using cache found in C:\Users\tan_l/.cache\torch\hub\pytorch_vision_v0.10.0


Started: 2024-04-29 06:39:24.651007
Learning Rate: 0.0001
Start Time : 1714343964.6510074
End Time : 1714350713.1347501
Time Taken: 6748.483742713928
Epoch 10
-------------------------------
Train Loss : tensor(4.4934, device='cuda:0', grad_fn=<NllLossBackward0>)
Test Avg Loss : 4.883101892471314
Test Accuracy : 11.333333333333332

-------------------------------
Done!


### DenseNET Model 201

In [ ]:
import torch

# input_batch = torch.stack(train_tensor_list)
# print(input_batch)

# device = (
#     "cuda"
#     if torch.cuda.is_available()
#     else "mps"
#     if torch.backends.mps.is_available()
#     else "cpu"
# )

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

# model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)
# # or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet169', pretrained=True)
model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet201', pretrained=True)
# # model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet161', pretrained=True)
# model.eval()

# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model.eval()

if torch.cuda.is_available():
    model.cuda()

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# with torch.no_grad():
#     output = model(input_batch)
# # Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
# print(output[0])
# # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
# probabilities = torch.nn.functional.softmax(output[0], dim=0)
# print(probabilities)

Using cuda device


Using cache found in C:\Users\tan_l/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\tan_l\anaconda3\envs\PyTorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to C:\Users\tan_l/.cache\torch\hub\checkpoints\densenet201-c1103571.pth
100%|██████████| 77.4M/77.4M [00:00<00:00, 94.6MB/s]


In [ ]:
optimizer_lr =[1e-2,1e-3,1e-4]

for j in optimizer_lr:
    model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet201', pretrained=True)
    model.train()
    if torch.cuda.is_available():
        model.cuda()


    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=j)

    def train(dataloader, model, loss_fn, optimizer):
        size = len(dataloader.dataset)
        model.train()
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            # Compute prediction error
            pred = model(X)
            loss = loss_fn(pred, y)

            # Backpropagation
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if batch % 100 == 0:
                loss, current = loss.item(), (batch + 1) * len(X)
                # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        return loss

    def test(dataloader, model, loss_fn):
        size = len(dataloader.dataset)
        num_batches = len(dataloader)
        model.eval()
        test_loss, correct = 0, 0
        with torch.no_grad():
            for X, y in dataloader:
                X, y = X.to(device), y.to(device)
                pred = model(X)
                test_loss += loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        # print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
        return (100*correct) , test_loss

    epochs = 10
    print("Started:", datetime.now())

    start = time.time()

    for t in range(epochs):

        loss = train(trainloader, model, loss_fn, optimizer)
        accuracy , avg_loss = test(testloader, model, loss_fn)
        if t == (epochs-1):
            end = time.time()
            print("Learning Rate:", j)
            print("Start Time :", start)
            print("End Time :", end)
            print("Time Taken:", end - start)
            print(f"Epoch {t+1}\n-------------------------------")
            print("Train Loss :", loss)
            print("Test Avg Loss :", avg_loss)
            print("Test Accuracy :", accuracy)
            print("\n-------------------------------")
print("Done!")

Using cache found in C:\Users\tan_l/.cache\torch\hub\pytorch_vision_v0.10.0


Started: 2024-04-29 09:37:29.297187


OutOfMemoryError: CUDA out of memory. Tried to allocate 392.00 MiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Of the allocated memory 21.03 GiB is allocated by PyTorch, and 1.38 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# epochs = 10
# print("Started:", datetime.now())
# for t in range(epochs):
#     print(f"Epoch {t+1}\n-------------------------------")
#     train(trainloader, model, loss_fn, optimizer)
#     test(testloader, model, loss_fn)
# print("Done!")

Started: 2024-04-08 02:52:52.648596
Epoch 1
-------------------------------
loss: 16.575052  [  128/33000]
loss: 0.512749  [12928/33000]
loss: 0.468490  [25728/33000]
Test Error: 
 Accuracy: 84.6%, Avg loss: 0.374671 

Epoch 2
-------------------------------
loss: 0.383459  [  128/33000]
loss: 0.224207  [12928/33000]
loss: 0.223989  [25728/33000]
Test Error: 
 Accuracy: 88.9%, Avg loss: 0.270216 

Epoch 3
-------------------------------
loss: 0.234675  [  128/33000]
loss: 0.234386  [12928/33000]
loss: 0.214994  [25728/33000]
Test Error: 
 Accuracy: 91.1%, Avg loss: 0.221622 

Epoch 4
-------------------------------
loss: 0.188882  [  128/33000]
loss: 0.222569  [12928/33000]
loss: 0.192542  [25728/33000]
Test Error: 
 Accuracy: 92.3%, Avg loss: 0.194899 

Epoch 5
-------------------------------
loss: 0.159839  [  128/33000]
loss: 0.164312  [12928/33000]
loss: 0.123608  [25728/33000]
Test Error: 
 Accuracy: 93.0%, Avg loss: 0.177767 

Epoch 6
-------------------------------
loss: 0.18102

In [ ]:
# # Save model

# print("Finished:", datetime.now())
# torch.save(model.state_dict(), "model.pth")
# print("Saved PyTorch Model State to model.pth")

Finished: 2024-04-08 18:30:15.324487
Saved PyTorch Model State to model.pth
